In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api=KEY, model_name="", temperature=0.5)

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=Template)

In [ ]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generative_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], 
                                            input_varables=["text", "number", "subject", "tone", "response_json"],
                                             output_variables=["quiz", "review"], verbose=True)

In [ ]:
filepath = "D:\Vishan\Generative_AI\MCQGen\data.txt"

In [ ]:
with open(filepath, 'r') as file:
    TEXT = file.read()

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "Machine learning"
TONE = "simple"

In [ ]:
with get_openai_callback() as cb:
    response = generative_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )